# Pulling Tweets from WHO

In [8]:
# Import libbraries
import os, sys, subprocess
import json
import pandas as pd
import numpy as np
from langdetect import detect
import pickle

pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 500)

In [13]:
# load project config

terminal_call = ! git rev-parse --show-toplevel
repo_path=terminal_call[0]
project_config_path = os.path.join(repo_path,'project_config.json')

with open(project_config_path,'r') as fp: 
    project_config = json.load(fp)


project_config

{'project_module_relative_path': 'src'}

In [14]:
# import custom scraper module

module_path = os.path.join(repo_path,project_config['project_module_relative_path'])
sys.path.append(module_path)

import scraper
from scraper import tweet_scraper

from importlib import reload

## Pull Tweets for a year

In [4]:
# get tweets from WHO for a year from this date
df = tweet_scraper.pull_tweets('WHO', '2019-06-01', '2020-06-01')

In [5]:
# rename columns
df.rename(columns={0:"date",
          1:"text"}, inplace = True)

In [6]:
# set column types
df['text'] = df['text'].astype(str)
df['date'] = pd.to_datetime(df['date'])

## Pickle for later use

In [29]:
pickle.dump(df, open("../data/who_tweets.pkl", "wb"))

## Clean Tweets

In [23]:
# Reload package when I make updates to function
# reload(tweet_scraper)

In [22]:
# Clean tweet column
tweet_scraper.clean_tweets(df, 'text')

,date,text
0,2020-05-31 19:50:07+00:00,there has never been a better time to quit smoking puts you at higher risk for a severe case of and a lot of other diseases that kill you
1,2020-05-31 19:23:26+00:00,quit now and breath easy within minutes of quitting your blood pressure drops after hours carbon monoxide levels in the bloodstream drops to normal within weeks your circulation improves and lung function increases
2,2020-05-31 19:00:46+00:00,on organize your own workshop to educate people around you spread awareness and create a tobacco free generation speak out workshop toolkit
3,2020-05-31 17:12:43+00:00,to help over billion tobacco users quit who is launching florence a digital health worker that combats misinformation about and tobacco and gives recommendations and tips for quitting
4,2020-05-31 16:41:11+00:00,quit today tips to curb your cravings delay as long as you can before giving in to your urge take deep breaths to relax yourself until the urge passes drinking is a healthy alternative do something else to distract yourself
...,...,...
6578,2019-06-03 12:57:48+00:00,well done you karolina enjoy
6579,2019-06-03 12:55:00+00:00,of road traffic deaths are pedestrians cyclists and motorcyclists
6581,2019-06-03 11:24:47+00:00,cycling is a simple affordable reliable clean and sustainable means of transportation that is both good for your health and the environment
6582,2019-06-03 10:46:48+00:00,cycling is a form of physical activity physical activity helps prevent heart disease stroke diabetes and some cancers and also helps prevent obesity and improves mental health and well being ride your bike to
